In [1]:
# import all necessary packages for webscraping a list of Bundesliga first devision teams

import time
import requests
from bs4 import BeautifulSoup
import re # support regular expressions
import pandas as pd
import numpy as np

In [2]:
# act as a browser to avoid the hp suspecting any scraping

headers = {'User-Agent': 
           'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari/537.36'}

In [3]:
# get content of the page that lists all current Bundeliga teams on tm.de

page = "https://www.transfermarkt.de/primera-division/startseite/wettbewerb/ES1"
page_tree = requests.get(page, headers=headers)
html_soup = BeautifulSoup(page_tree.content, 'html.parser')

In [4]:
# extracting table from the soup object
club_table = html_soup.find(id='yw1')

In [5]:
club_name_elements = club_table.find_all('td', {'class': 'hauptlink no-border-links'})
club_names = [club.text.replace(u'\xa0', u'').strip() for club in club_name_elements]
club_names

['Real Madrid',
 'FC Barcelona',
 'Atlético Madrid',
 'Real Sociedad San Sebastián',
 'FC Villarreal',
 'Real Betis Sevilla',
 'FC Valencia',
 'FC Sevilla',
 'Athletic Bilbao',
 'FC Getafe',
 'Celta Vigo',
 'CA Osasuna',
 'FC Girona',
 'Espanyol Barcelona',
 'UD Almería',
 'Rayo Vallecano',
 'RCD Mallorca',
 'FC Elche',
 'Real Valladolid',
 'FC Cádiz']

In [6]:
team_size_elements = club_table.find_all('td', {'class': 'zentriert'})
team_sizes = [size.text for size in team_size_elements][4::4]
team_sizes

['24',
 '22',
 '22',
 '26',
 '22',
 '26',
 '25',
 '24',
 '25',
 '25',
 '23',
 '21',
 '25',
 '27',
 '27',
 '25',
 '26',
 '26',
 '24',
 '29']

In [7]:
# extracting the average age from club_table

avg_team_age_elements = club_table.find_all('td', {'class': 'zentriert'})
avg_team_ages = [age.text for age in avg_team_age_elements][5::4]
avg_team_ages

['27,6',
 '25,3',
 '28,4',
 '25,5',
 '28,4',
 '28,7',
 '25,0',
 '28,6',
 '27,6',
 '27,4',
 '27,0',
 '28,0',
 '26,4',
 '25,7',
 '25,7',
 '28,5',
 '27,4',
 '27,6',
 '27,8',
 '28,7']

In [8]:
# extracting the average value of the teams players

avg_team_value_elements = club_table.find_all('td', {'class': 'rechts'})
avg_team_values = [float(value.text.split()[0].replace(',' , '.')) for value in avg_team_value_elements][2::2]
avg_team_values

[35.46,
 34.64,
 22.23,
 14.63,
 13.17,
 10.1,
 9.98,
 10.0,
 8.87,
 5.69,
 5.46,
 5.91,
 4.59,
 4.01,
 3.54,
 3.53,
 3.31,
 2.66,
 2.83,
 2.13]

In [9]:
# extracting each teams total value from club_table

# finding each teams total value and put in a list
avg_team_value_elements = club_table.find_all('td', {'class': 'rechts'})
team_values = [float(value.text.split()[0].replace(',' , '.')) for value in avg_team_value_elements][3::2]
team_values

[851.0,
 762.0,
 489.0,
 380.5,
 289.7,
 262.5,
 249.5,
 240.0,
 221.8,
 142.2,
 125.5,
 124.2,
 114.7,
 108.4,
 95.7,
 88.2,
 86.1,
 69.1,
 67.8,
 61.9]

In [11]:
# creating a pandas data frame of the tm.de table and zip team names with the squad size

df = pd.DataFrame(list(zip(club_names, team_sizes)),columns=['clubs', 'squad'])
df

,clubs,squad
0,Real Madrid,24
1,FC Barcelona,22
2,Atlético Madrid,22
3,Real Sociedad San Sebastián,26
4,FC Villarreal,22
5,Real Betis Sevilla,26
6,FC Valencia,25
7,FC Sevilla,24
8,Athletic Bilbao,25
9,FC Getafe,25


In [15]:
# adding the columns 'avg_age', 'avg_market_value', 'market value'
# populate the values of their respective lists as Pandas Series

df['avg_age'] = pd.Series(avg_team_ages)
df['avg_market_value'] = pd.Series(avg_team_values)
df['market_value'] = pd.Series(team_values)
df

,clubs,squad,avg_age,avg_market_value,market_value
0,Real Madrid,24,"27,6",35.46,851.0
1,FC Barcelona,22,"25,3",34.64,762.0
2,Atlético Madrid,22,"28,4",22.23,489.0
3,Real Sociedad San Sebastián,26,"25,5",14.63,380.5
4,FC Villarreal,22,"28,4",13.17,289.7
5,Real Betis Sevilla,26,"28,7",10.10,262.5
6,FC Valencia,25,"25,0",9.98,249.5
7,FC Sevilla,24,"28,6",10.00,240.0
8,Athletic Bilbao,25,"27,6",8.87,221.8
9,FC Getafe,25,"27,4",5.69,142.2


In [23]:
# add country column an populate String value La Liga


df['country'] = pd.Series(['ES' for club in club_names])
df

,clubs,squad,avg_age,avg_market_value,market_value,country
0,Real Madrid,24,"27,6",35.46,851.0,ES
1,FC Barcelona,22,"25,3",34.64,762.0,ES
2,Atlético Madrid,22,"28,4",22.23,489.0,ES
3,Real Sociedad San Sebastián,26,"25,5",14.63,380.5,ES
4,FC Villarreal,22,"28,4",13.17,289.7,ES
5,Real Betis Sevilla,26,"28,7",10.10,262.5,ES
6,FC Valencia,25,"25,0",9.98,249.5,ES
7,FC Sevilla,24,"28,6",10.00,240.0,ES
8,Athletic Bilbao,25,"27,6",8.87,221.8,ES
9,FC Getafe,25,"27,4",5.69,142.2,ES
